In [70]:
print("OK")

OK


In [71]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
from dotenv import load_dotenv
import pinecone
import uuid
import math
import json

In [72]:
load_dotenv()

True

In [73]:
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV='gcp-starter'

In [74]:
# Extract data from PDF file
def load_pdf(data):
 loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
 documents=loader.load()
 return documents

In [75]:
extracted_data = load_pdf("data/")

In [76]:
#extracted_data

In [77]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [78]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [79]:
text_chunks[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [80]:
#text_chunks

In [81]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [82]:
embedding_model = download_hugging_face_embeddings()

g:\End-to-end-medical-chatbot-using-llama2\mchatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [83]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [84]:
query_result = embedding_model.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [85]:
#query_result

In [86]:
import os
from pinecone import Pinecone, ServerlessSpec
#from langchain.embeddings.openai import OpenAIEmbeddings

In [87]:
# Initialize Pinecone
api_key = os.getenv("PINECONE_API_KEY")

In [88]:
# Define index name
index_name = "medical-chatbot"
pc = Pinecone(api_key=api_key)
# Creates an index using the API key stored in the client 'pc'.
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
) 

In [89]:
# Assuming 'text_chunks' is a list of objects where each object has a 'page_content' attribute
#Example: text_chunks = [TextChunk(page_content="Text 1"), TextChunk(page_content="Text 2")]
# Generate embeddings for each text chunk
embeddings = [embedding_model.embed_query(t.page_content) for t in text_chunks]

In [90]:
len(embeddings)

7020

In [91]:
embeddings[0]

[0.0017460824456065893,
 -0.03350288048386574,
 -0.03290391340851784,
 0.007168074604123831,
 -0.01460330095142126,
 0.010261906310915947,
 -0.011515316553413868,
 0.22930210828781128,
 -0.023232368752360344,
 0.004120390862226486,
 -0.036560822278261185,
 0.08592111617326736,
 0.012972141616046429,
 0.05221789330244064,
 -0.10232619196176529,
 -0.0031390218064188957,
 -0.012686936184763908,
 0.00047184049617499113,
 -0.028485851362347603,
 -0.05025918409228325,
 0.01155098993331194,
 0.0778065174818039,
 0.09282821416854858,
 -0.013797298073768616,
 -0.016935091465711594,
 -0.02595585398375988,
 -0.04956509545445442,
 -0.046131327748298645,
 0.007290528621524572,
 -0.013553302735090256,
 0.03843941166996956,
 0.06280472129583359,
 0.01835383102297783,
 0.008242791518568993,
 0.0017155827954411507,
 -0.03986185044050217,
 -0.01163862831890583,
 0.016446184366941452,
 0.025595610961318016,
 0.09104608744382858,
 0.02967270091176033,
 -0.054160282015800476,
 -0.04576560854911804,
 -0.013

In [92]:
index_name

'medical-chatbot'

In [93]:
index = pc.Index(index_name)

In [94]:
index

In [95]:
# Retrieve index info to get the host
index_info = pc.describe_index(index_name)

In [96]:
index_info

{'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'medical-chatbot-omc9wav.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'medical-chatbot',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [97]:
host=index_info['host']
print(host)

medical-chatbot-omc9wav.svc.aped-4627-b74a.pinecone.io


In [98]:
def flatten_metadata(metadata):
    """Flatten metadata to simple key-value pairs."""
    flattened = {}
    for key, value in metadata.items():
        if isinstance(value, (str, int, float, bool)):
            flattened[key] = value
        elif isinstance(value, list) and all(isinstance(i, str) for i in value):
            flattened[key] = value
        else:
            # Convert complex structures to JSON strings
            flattened[key] = json.dumps(value)
    return flattened

In [99]:
# Helper function to split vectors into smaller batches
def batch_vectors(vectors, batch_size):
    for i in range(0, len(vectors), batch_size):
        yield vectors[i:i + batch_size]

In [101]:
# Define batch size (adjust as needed)
batch_size = 1000  # Set a batch size that fits within Pinecone's limits

# Prepare the batch of vectors to upsert
vectors = [(str(uuid.uuid4()), list(embedding), {
    "text": chunk.page_content,
    **flatten_metadata(chunk.metadata),
}) for embedding, chunk in zip(embeddings, text_chunks)]
namespace="ns_medical_chatbot"
# Upsert vectors in batches
for batch in batch_vectors(vectors, batch_size):
    index.upsert(vectors=batch,namespace=namespace)
    print(f"Uploaded {len(batch)} vectors to Pinecone index '{index}'")

print(f"Uploaded a total of {len(vectors)} vectors to Pinecone index '{index}'")


Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 1000 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded 20 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
Uploaded a total of 7020 vectors to Pinecone index '<pinecone.data.index.Index object at 0x000001A7CD7126D0>'
